<a href="https://colab.research.google.com/github/HarryKao1020/stock_trading_strategy/blob/colab/basic/tick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib


!pip install backtesting
!pip install shioaji
!pip install python-dotenv
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
# import talib
import os
from dotenv import load_dotenv

from google.colab import userdata

# 加載 .env 文件中的環境變數
load_dotenv()

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for jupyter
# shioaji_secret = os.getenv("SHIOAJI_SECRETKEY", None)
# shioaji_apikey = os.getenv("SHIOAJI_APIKEY", None)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')



# 登入你的key
# accounts = api.login("YOUR_API_KEY", "YOUR_SECRET_KEY")
accounts = api.login(shioaji_apikey, shioaji_secret)

--2024-08-23 09:35:59--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-08-23 09:36:00--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-08-23 09:36:00--  http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1
Res

2024-08-23 09:36:54.044 | WARNING  | importlib._bootstrap:_call_with_frames_removed:241 - Optional: pip install shioaji[speed] for better performance.
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [12]:
ticks = api.ticks(
    contract=api.Contracts.Stocks["5443"],
    date="2024-08-23",
    query_type=sj.constant.TicksQueryType.RangeTime,
    time_start="09:14:00",
    time_end="09:20:00"
)
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df

,ts,close,volume,bid_price,bid_volume,ask_price,ask_volume,tick_type
0,2024-08-23 09:14:02.879886,117.5,20,117.0,81,117.5,89,1
1,2024-08-23 09:14:03.136135,117.0,1,117.0,82,117.5,92,2
2,2024-08-23 09:14:03.196541,117.0,1,117.0,80,117.5,91,2
3,2024-08-23 09:14:04.493840,117.0,3,117.0,78,117.5,91,2
4,2024-08-23 09:14:04.593052,117.5,1,117.0,77,117.5,90,1
...,...,...,...,...,...,...,...,...
368,2024-08-23 09:19:46.578047,117.0,5,117.0,63,117.5,119,2
369,2024-08-23 09:19:51.790125,117.5,1,117.0,67,117.5,118,1
370,2024-08-23 09:19:51.822976,117.5,118,117.0,67,118.0,178,1
371,2024-08-23 09:19:53.264825,117.5,20,117.5,18,118.0,180,2


from matplotlib import pyplot as plt
df['close'].plot(kind='hist', bins=20, title='close')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['volume'].plot(kind='hist', bins=20, title='volume')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bid_price'].plot(kind='hist', bins=20, title='bid_price')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bid_volume'].plot(kind='hist', bins=20, title='bid_volume')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['ts']
  ys = series['bid_volume']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('ts', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ts')
_ = plt.ylabel('bid_volume')

from matplotlib import pyplot as plt
df['close'].plot(kind='line', figsize=(8, 4), title='close')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['volume'].plot(kind='line', figsize=(8, 4), title='volume')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['bid_price'].plot(kind='line', figsize=(8, 4), title='bid_price')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['bid_volume'].plot(kind='line', figsize=(8, 4), title='bid_volume')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
big_volume_df = df[['bid_volume']>300]